In [ ]:
#The problem here is , name:str, age:int, they are just type hints, which lets the developers know what kind
#of value is expected in these fields, it is not an enforcement of any constraint, that will throw error if 
#the specified value is not passed.
#in the below name is str, but i have passed float value in arguments, and it happily accepted it, which is the main problem
#where we are going to use pydantic.
class A:
    def __init__(self, name: str, age: int):
        self.name = name
        self.age = age

a = A(12.22, (22))
print(type(a.name), a.age)


<class 'float'> 22


In [ ]:
#its not a best way to use BaseModel, cause when use __init__, we are oveririding the init method inside the pydantic module.
from pydantic import BaseModel
class A(BaseModel):
    def __init__(self, name: str, age: int):
        self.name = name
        self.age = age

a = A('ajay', (22))


ValueError: "A" object has no field "name"

In [ ]:
#BaseModel.__init__() takes 1 positional argument but 3 were given
#this error means, that it expects keyword arguments after the 1st positional argument which is by default self in the init method.
from pydantic import BaseModel
class A(BaseModel):
    name: str
    age: int

a = A('ajay', 22)
a

TypeError: BaseModel.__init__() takes 1 positional argument but 3 were given

In [ ]:
#correct scenario
from pydantic import BaseModel
class A(BaseModel):
    name: str
    age: int

a = A(name = 'ajay',age= 22)
a

A(name='ajay', age=22)

In [19]:
#lets pass a wrong data type value in name
from pydantic import BaseModel
class A(BaseModel):
    name: str
    age: int

a = A(name = 12,age= 22)
a

ValidationError: 1 validation error for A
name
  Input should be a valid string [type=string_type, input_value=12, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type

In [23]:
#Now instead of just defining data types, we can also define whether the field is Optional, as well as default value
from typing import Optional
class A(BaseModel):
    name: str
    age: int
    place: Optional[str] = 'Delhi'

a = A(name = 'ajay',age= 22)
print(a)

b = A(name = 'ajay',age= 22, place = 'Mumbai')
print(b)

name='ajay' age=22 place='Delhi'
name='ajay' age=22 place='Mumbai'


In [25]:
#we can also enforce if the data type is a List
from typing import List
class ClassRoom(BaseModel):
    name:str
    age:int
    marks: List[float]

#tuple is passed, implicit type casting can take place, which is not an issue
students = ClassRoom(name = 'abhishek', age = 12, marks = (40,80,90))
students

ClassRoom(name='abhishek', age=12, marks=[40.0, 80.0, 90.0])

In [ ]:
#lets handle the error of pydantic with try and except
from pydantic import ValidationError

try:
    student2 = ClassRoom(name=12, age=12, marks=(40,80,90))
except ValueError as e:
    print("Validation error occurred!")
    print(e)  # This prints full details of what went wrong

Validation error occurred!
1 validation error for ClassRoom
name
  Input should be a valid string [type=string_type, input_value=12, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type


In [34]:
#complex structure with nested model
class Address(BaseModel):
    city:str
    state:str
    pincode:int

class Students(BaseModel):
    name:str
    age:int
    address:Address

s = Students(name = "lalu", age = 20, address={'city':'Patna', 'state':'Bihar', 'pincode':789123})
s

Students(name='lalu', age=20, address=Address(city='Patna', state='Bihar', pincode=789123))

In [42]:
#beyond basic data type constainst, we can add other constraints as well
from pydantic import Field
class A(BaseModel):
    name: str=Field(min_length=2, max_length = 20) # min and max length of value
    age: int=Field(gt=3,le=25) #  3 < age <= 25

a = A(name = 'ajats',age= 4)
a

A(name='ajats', age=4)

In [43]:
b = A(name = 'ajats',age=3)
b

ValidationError: 1 validation error for A
age
  Input should be greater than 3 [type=greater_than, input_value=3, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/greater_than

In [44]:
c = A(name = 'ajatssssssssssssssssssssssssssssssss',age= 4)
c

ValidationError: 1 validation error for A
name
  String should have at most 20 characters [type=string_too_long, input_value='ajatssssssssssssssssssssssssssssssss', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/string_too_long

In [ ]:
#we can also provide description
#... 3 dots means , the field is required.
#purpose of providng desccription, is for developers to know the usage of each field just by printing the schema
#default_factory is used when we want to set the values returned by a fucntion
class User(BaseModel):
    username: str = Field(...,description='unique user name for the user')
    age: int = Field(default=18, description = 'User age, defaults to 18')
    email: str = Field(default_factory=lambda:"dummyuser@example.com",description='default email address')

user1 = User(username = 'alice')
print(user1.model_json_schema())

{'properties': {'username': {'description': 'unique user name for the user', 'title': 'Username', 'type': 'string'}, 'age': {'default': 18, 'description': 'User age, defaults to 18', 'title': 'Age', 'type': 'integer'}, 'email': {'description': 'default email address', 'title': 'Email', 'type': 'string'}}, 'required': ['username'], 'title': 'User', 'type': 'object'}
